<a href="https://colab.research.google.com/github/pasquale90/pi-loop-console/blob/dev-v.0.2/d2m_groove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
gdrive='/content/gdrive'
drive.mount(gdrive, force_remount=True)

Mounted at /content/gdrive


#Download dataset

In [ ]:
'''
!pwd
%cd /content/gdrive/MyDrive
!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0.zip
#!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
!unzip groove-v1.0.0.zip
!rm groove-v1.0.0.zip
!ls
datapath="/content/gdrive/MyDrive/groove"
'''

'\n!pwd\n%cd /content/gdrive/MyDrive\n!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0.zip\n#!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip\n!unzip groove-v1.0.0.zip\n!rm groove-v1.0.0.zip\n!ls\ndatapath="/content/gdrive/MyDrive/groove"\n'

In [ ]:
''' code provided by authors
import tensorflow as tf
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
tf.executing_eagerly()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]
  print(f' Midi\n{midi}')
  print(f' Genre\n{genre}')
'''

In [ ]:
'''alternative dataset - midi only
!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
!unzip groove-v1.0.0-midionly.zip
!rm groove-v1.0.0-midionly.zip
!ls
'''

In [ ]:
#!cp -r /content/sample_data/groove /content/gdrive/MyDrive/

#Familiarization with mido

In [2]:
!pip install mido==1.2.9

     |████████████████████████████████| 52 kB 1.0 MB/s 


In [3]:
import os
import mido #import MidiFile, tempo2bpm, tick2second
import pandas as pd
import librosa

datapath="/content/gdrive/MyDrive/groove/"

In [4]:
sample_midi_path=os.path.join(datapath,'drummer1/session1/106_funk_95_fill_4-4.mid')
mid = mido.MidiFile(sample_midi_path)#, clip=True
print(mid,"\n")

for i,track in enumerate(mid.tracks):
    print(f'track_{i}:\n {track}')

<midi file '/content/gdrive/MyDrive/groove/drummer1/session1/106_funk_95_fill_4-4.mid' type 0, 1 tracks, 84 messages> 

track_0:
 <midi track '' 84 messages>


# define classes

In [5]:
classes={"Bass":36,
        "Snare":38,
        "High_Tom":50,
        "Low_Mid_Tom":47,
        "High_Floor_Tom":43,
        "Open_Hi_Hat":46,
        "Closed_Hi_Hat":42,
        "Crash_Cymbal":49,
        "Ride_Cymbal":51}

def get_class_string(id):
  return [k for k,v in classes.items() if v == id][0]

print(get_class_string(36))

'''
classes=["Bass",
        "Snare",
        "High_Tom",
        "Low_Mid_Tom",
        "High_Floor_Tom",
        "Open_Hi_Hat",
        "Closed_Hi_Hat",
        "Crash_Cymbal",
        "Ride_Cymbal"]

def get_class_id(class_str):
  return [i for i,c in enumerate(classes) if c==class_str][0]

def get_class_given_id(id):
  return classes[id]

id=get_class_id("Ride_Cymbal")
print(id)
print(get_class_given_id(id))
'''
#print(classes)

Bass


'\nclasses=["Bass",\n        "Snare",\n        "High_Tom",\n        "Low_Mid_Tom",\n        "High_Floor_Tom",\n        "Open_Hi_Hat",\n        "Closed_Hi_Hat",\n        "Crash_Cymbal",\n        "Ride_Cymbal"]\n\ndef get_class_id(class_str):\n  return [i for i,c in enumerate(classes) if c==class_str][0]\n\ndef get_class_given_id(id):\n  return classes[id]\n\nid=get_class_id("Ride_Cymbal")\nprint(id)\nprint(get_class_given_id(id))\n'

# design Note and Midifile classes

In [ ]:
'''
class Midifile:
  def __init__(self, filepath):
    self.midifile=mido.MidiFile(filepath)
    self.tempo=0
    self.bpm=0
    self.rythm_numerator=0
    self.rythm_denominator=0

    self.notes=[]

    self.PPQ=self.midifile.ticks_per_beat
    self.total_ticks=self.set_length_ticks()
    self.ending_tick=0

    #not explicitly required yet
    self.key=''
    self.clocks_per_click=0
    self.notated_32nd_notes_per_beat=0


  def get_midifile(self):
    return self.midifile

  def set_length_ticks(self):
    elapsed_ticks=0
    for track in self.midifile.tracks[0]:
      elapsed_ticks+=track.time
    self.total_ticks=elapsed_ticks

  def get_total_ticks(self):
    return self.total_ticks
  
  def get_total_seconds(self):
    return mido.tick2second(self.total_ticks,self.PPQ,self.tempo)

  def parse_midi_messages(self):

    for message in self.midifile.tracks[0]:
      message_type=message.type

      if message_type=="set_tempo":
        self.tempo=message.tempo
        self.bmp=int(mido.tempo2bpm(self.tempo))

      elif message_type=="time_signature":
        self.rythm_numerator=message.numerator
        self.rythm_denominator=message.denominator
        self.clocks_per_click=message.clocks_per_click
        self.notated_32nd_notes_per_beat=message.notated_32nd_notes_per_beat
        
      elif message_type=="key_signature":
        self.key=message.key
      
      elif message_type=="end_of_track":
        self.ending_tick=message.time
     
      else:
        #print(f"Unknown message {message}")
        pass
      
      #'' Initialize Notes and add them on the self.notes list - ignore program_change and control_change_messages
      elif message_type=="program_change":
        action="program_change??"
        channel=message.channel
        program=message.program 
        time=message.time
      
      elif message_type=="control_change":
        action="control_change??"
        channel=message.channel
        control=message.control
        value=message.value
        time=message.time
    
      elif message_type=="note_on":
        action="new_note"
        channel=message.channel 
        note=message.note
        note=remap_pitch(note)
        velocity=message.velocity
        time=message.time
      #''
      
  def get_ms_per_tick(self):
    return (60000 / (self.bpm * self.PPQ))
'''

In [46]:
#make a Note class
class Note:
  def __init__(self,channel,pitch,velocity,timestamp,duration):
    self.channel=channel
    self.pitch=self.remap_pitch(pitch)
    self.label=self.set_class(self.pitch)
    self.velocity=velocity
    self.timestamp=timestamp
    self.duration=duration
    
    self.relative_time=0 #in which 
    
  
  def get_label(self):
    return self.label
  def get_pitch(self):
    return self.pitch
  def get_duration(self):
    return self.duration
  def get_velocity(self):
    return self.velocity
  def get_timestamp(self):
    return self.timestamp

  def remap_pitch(self,pitch):
    if pitch==36:
      return 36#"Bass"
    elif pitch in (37,38,40):
      return 38#"Snare"
    elif pitch in (48,50):
      return 50#"High_Tom"
    elif pitch in (45,47):
      return 47#"Low_Mid_Tom"
    elif pitch in (43,58):
      return 43#"High_Floor_Tom"
    elif pitch in (26,46):
      return 46#"Open_Hi_Hat"
    elif pitch in (22,42,44):
      return 42#"Closed_Hi_Hat"
    elif pitch in (49,52,55,57):
      return 49#"Crash_Cymbal"
    elif pitch in (51,53,59):
      return 51#"Ride_Cymbal"
  
  def set_class(self,id):
    classes={"Bass":36,
        "Snare":38,
        "High_Tom":50,
        "Low_Mid_Tom":47,
        "High_Floor_Tom":43,
        "Open_Hi_Hat":46,
        "Closed_Hi_Hat":42,
        "Crash_Cymbal":49,
        "Ride_Cymbal":51}

    return [k for k,v in classes.items() if v == id][0]
  
  def tick2second(tick, ticks_per_beat, tempo):
    """Convert absolute time in ticks to seconds.
    Returns absolute time in seconds for a chosen MIDI file time
    resolution (ticks per beat, also called PPQN or pulses per quarter
    note) and tempo (microseconds per beat).
    """
    scale = tempo * 1e-6 / ticks_per_beat
    return tick * scale

In [44]:
class Midifile:
  def __init__(self,filepath):
    self.midifile=mido.MidiFile(filepath)
    self.PPQ=self.midifile.ticks_per_beat
    self.notes=[]
    
    elapsed_ticks=0
    for m,message in enumerate(self.midifile.tracks[0]):
      message_type=message.type
      
      elapsed_ticks+=message.time
      print(f'message {m} , time= {message.time} elapsed_ticks={elapsed_ticks}')

      if message_type=="set_tempo":
        self.tempo=message.tempo
        self.bpm=int(mido.tempo2bpm(self.tempo))

      elif message_type=="time_signature":
        self.rythm_numerator=message.numerator
        self.rythm_denominator=message.denominator
        self.clocks_per_click=message.clocks_per_click
        self.notated_32nd_notes_per_beat=message.notated_32nd_notes_per_beat
        
      elif message_type=="key_signature":
        self.key=message.key
      
      elif message_type=="end_of_track":
        self.ending_tick=message.time
        self.total_ticks=elapsed_ticks

      elif message_type=="note_on":
        channel=message.channel 
        pitch=message.note
        velocity=message.velocity
        time=message.time
        #duration=self.midifile.tracks[0][m+1].time
        #print(f'duration={self.midifile.tracks[0][m+1].time}')
        timestamp=elapsed_ticks
        if velocity!=0:
          temp_counter=1
          duration=0
          for msg in self.midifile.tracks[0][m+1:]:
            duration+=msg.time
            if msg.type=="note_on" and msg.note==pitch and msg.velocity==0:
              new_note=Note(channel,pitch,velocity,timestamp,duration)
              self.notes.append(new_note)
              break
            else:
              print('NoteDurationError')
      else:
        #print(f"Unknown message {message}")
        pass

    

    #self.total_ticks=self.set_length_ticks()
    #self.ending_tick=0

   #not explicitly required yet
    #self.key=''
    #self.clocks_per_click=0
    #self.notated_32nd_notes_per_beat=0

  def get_midifile(self):
    return self.midifile
  def get_tempo(self):
    return self.tempo
  def get_bpm(self):
    return self.bpm
  def get_rythm_numerator(self):
    return self.rythm_numerator
  def get_rythm_denominator(self):
    return self.rythm_denominator
  
  def get_note(self,idx):
    return self.notes[idx]
  
  def get_num_notes(self):
    return len(self.notes)

  def get_total_ticks(self):
    return self.total_ticks
  
  def get_total_seconds(self):
    return mido.tick2second(self.total_ticks,self.PPQ,self.tempo)

  def parse_midi_messages(self):

    for message in self.midifile.tracks[0]:
      message_type=message.type

      if message_type=="set_tempo":
        self.tempo=message.tempo
        self.bmp=int(mido.tempo2bpm(self.tempo))

      elif message_type=="time_signature":
        self.rythm_numerator=message.numerator
        self.rythm_denominator=message.denominator
        self.clocks_per_click=message.clocks_per_click
        self.notated_32nd_notes_per_beat=message.notated_32nd_notes_per_beat
        
      elif message_type=="key_signature":
        self.key=message.key
      
      elif message_type=="end_of_track":
        self.ending_tick=message.time
     
      else:
        #print(f"Unknown message {message}")
        pass
      
      ''' Initialize Notes and add them on the self.notes list - ignore program_change and control_change_messages
      elif message_type=="program_change":
        action="program_change??"
        channel=message.channel
        program=message.program 
        time=message.time
      
      elif message_type=="control_change":
        action="control_change??"
        channel=message.channel
        control=message.control
        value=message.value
        time=message.time
    
      elif message_type=="note_on":
        action="new_note"
        channel=message.channel 
        note=message.note
        note=remap_pitch(note)
        velocity=message.velocity
        time=message.time
      '''
      
  def get_ms_per_tick(self):
    return (60000 / (self.bpm * self.PPQ))

In [47]:
test_midi=Midifile(sample_midi_path)
test_midi.parse_midi_messages()
print(test_midi.get_tempo())
print(test_midi.get_bpm())

message 0 , time= 0 elapsed_ticks=0
message 1 , time= 0 elapsed_ticks=0
message 2 , time= 0 elapsed_ticks=0
message 3 , time= 0 elapsed_ticks=0
message 4 , time= 0 elapsed_ticks=0
message 5 , time= 0 elapsed_ticks=0
message 6 , time= 204 elapsed_ticks=204
message 7 , time= 203 elapsed_ticks=407
message 8 , time= 203 elapsed_ticks=610
message 9 , time= 203 elapsed_ticks=813
message 10 , time= 205 elapsed_ticks=1018
message 11 , time= 203 elapsed_ticks=1221
message 12 , time= 203 elapsed_ticks=1424
message 13 , time= 203 elapsed_ticks=1627
message 14 , time= 196 elapsed_ticks=1823
NoteDurationError
message 15 , time= 8 elapsed_ticks=1831
message 16 , time= 37 elapsed_ticks=1868
message 17 , time= 0 elapsed_ticks=1868
message 18 , time= 31 elapsed_ticks=1899
message 19 , time= 50 elapsed_ticks=1949
message 20 , time= 77 elapsed_ticks=2026
message 21 , time= 8 elapsed_ticks=2034
message 22 , time= 16 elapsed_ticks=2050
message 23 , time= 77 elapsed_ticks=2127
message 24 , time= 33 elapsed_

In [48]:
num=test_midi.get_num_notes()
for i in range(num):
  n=test_midi.get_note(i)
  print(f'note {i+1}--> label:{n.get_label()}, timestamp:{n.get_timestamp()}/{test_midi.get_total_ticks()}, duration: {n.get_duration()}, pitch:{n.get_pitch()}, velocity:{n.get_velocity()}')

note 1--> label:Snare, timestamp:1823/4561, duration: 45, pitch:38, velocity:21
note 2--> label:Snare, timestamp:1868/4561, duration: 31, pitch:38, velocity:38
note 3--> label:Snare, timestamp:1949/4561, duration: 77, pitch:38, velocity:88
note 4--> label:Snare, timestamp:2050/4561, duration: 77, pitch:38, velocity:81
note 5--> label:Snare, timestamp:2160/4561, duration: 77, pitch:38, velocity:56
note 6--> label:Snare, timestamp:2272/4561, duration: 76, pitch:38, velocity:74
note 7--> label:Snare, timestamp:2386/4561, duration: 77, pitch:38, velocity:67
note 8--> label:Bass, timestamp:2492/4561, duration: 77, pitch:36, velocity:52
note 9--> label:Open_Hi_Hat, timestamp:2509/4561, duration: 77, pitch:46, velocity:83
note 10--> label:Closed_Hi_Hat, timestamp:2596/4561, duration: 77, pitch:42, velocity:85
note 11--> label:High_Tom, timestamp:2737/4561, duration: 76, pitch:50, velocity:92
note 12--> label:High_Tom, timestamp:2854/4561, duration: 77, pitch:50, velocity:98
note 13--> label:H

# read midi files

In [ ]:
#read midi files using info.csv
csvpath=os.path.join(datapath,'info.csv')
csvfile=pd.read_csv(csvpath,header=None)
print(csvfile)
print(type(csvfile))

            0                      1                         2   \
0      drummer                session                        id   
1     drummer1  drummer1/eval_session   drummer1/eval_session/1   
2     drummer1  drummer1/eval_session  drummer1/eval_session/10   
3     drummer1  drummer1/eval_session   drummer1/eval_session/2   
4     drummer1  drummer1/eval_session   drummer1/eval_session/3   
...        ...                    ...                       ...   
1146  drummer2      drummer2/session2      drummer2/session2/11   
1147  drummer2      drummer2/session2      drummer2/session2/12   
1148  drummer2      drummer2/session2      drummer2/session2/13   
1149  drummer2      drummer2/session2      drummer2/session2/14   
1150  drummer2      drummer2/session2      drummer2/session2/15   

                 3    4          5               6   \
0             style  bpm  beat_type  time_signature   
1      funk/groove1  138       beat             4-4   
2     soul/groove10  102      

In [ ]:
midipaths=[]
#read midi files by iteratin through the folders
for folder in os.listdir(datapath):
  print (f'folder {folder}')
  folderpath=os.path.join(datapath,folder)
  if os.path.isdir(folderpath):
    for session_folder in os.listdir(folderpath):
      print(f'\tsession_folder {session_folder}')
      session_folderpath=os.path.join(folderpath,session_folder)
      if os.path.isdir(session_folderpath):
        for midifile in os.listdir(session_folderpath):
          #print('\t\t\t',midifile)
          midipath=os.path.join(midifile,session_folder)
          if midifile[-4:]=='.mid':
            midipaths.append(midipath)
print(f'{len(midipaths)} midifiles are found')

folder README
folder info.csv
folder Icon
folder drummer1
	session_folder Icon
	session_folder eval_session
	session_folder session2
	session_folder session1
	session_folder session3
folder drummer10
	session_folder Icon
	session_folder session1
folder drummer3
	session_folder Icon
	session_folder session2
	session_folder session1
folder drummer2
	session_folder Icon
	session_folder session1
	session_folder session2
	session_folder session3
folder drummer4
	session_folder Icon
	session_folder session1
folder drummer5
	session_folder Icon
	session_folder eval_session
	session_folder session1
	session_folder session2
folder drummer7
	session_folder Icon
	session_folder eval_session
	session_folder session1
	session_folder session2
	session_folder session3
folder drummer6
	session_folder Icon
	session_folder session1
	session_folder session2
	session_folder session3
folder drummer8
	session_folder Icon
	session_folder eval_session
	session_folder session1
	session_folder session2
folder d

In [ ]:
elapsed_ticks=0
for i,track in enumerate(mid.tracks[0]):
  method,values,ticks_passed=parse_midi_message(track)
  elapsed_ticks+=ticks_passed
  if method=="new_note":
    print(f'note:{get_class_string(values[1])},velocity:{values[2]},time:{values[3]}')

print(f' Ticks elapsed = {elapsed_ticks}')

note:Snare,velocity:21,time:196
note:Snare,velocity:0,time:37
note:Snare,velocity:38,time:0
note:Snare,velocity:0,time:31
note:Snare,velocity:88,time:50
note:Snare,velocity:0,time:77
note:Snare,velocity:81,time:16
note:Snare,velocity:0,time:77
note:Snare,velocity:56,time:33
note:Snare,velocity:0,time:0
note:Snare,velocity:74,time:17
note:Snare,velocity:0,time:63
note:Snare,velocity:67,time:13
note:Snare,velocity:0,time:2
note:Bass,velocity:52,time:29
note:Open_Hi_Hat,velocity:83,time:17
note:Bass,velocity:0,time:19
note:Open_Hi_Hat,velocity:0,time:17
note:Closed_Hi_Hat,velocity:85,time:10
note:Closed_Hi_Hat,velocity:0,time:36
note:High_Tom,velocity:92,time:12
note:High_Tom,velocity:0,time:76
note:High_Tom,velocity:98,time:40
note:High_Tom,velocity:0,time:11
note:High_Tom,velocity:103,time:4
note:High_Tom,velocity:0,time:8
note:High_Tom,velocity:101,time:14
note:High_Tom,velocity:0,time:54
note:Bass,velocity:51,time:29
note:Bass,velocity:0,time:76
note:High_Floor_Tom,velocity:127,time:5

In [ ]:
for msg in mid.tracks[0]:
    print(msg)

<meta message set_tempo tempo=631578 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
program_change channel=9 program=0 time=0
control_change channel=9 control=4 value=89 time=0
program_change channel=9 program=0 time=0
control_change channel=9 control=4 value=88 time=204
control_change channel=9 control=4 value=87 time=203
control_change channel=9 control=4 value=86 time=203
control_change channel=9 control=4 value=85 time=203
control_change channel=9 control=4 value=84 time=205
control_change channel=9 control=4 value=83 time=203
control_change channel=9 control=4 value=82 time=203
control_change channel=9 control=4 value=81 time=203
note_on channel=9 note=38 velocity=21 time=196
control_change channel=9 control=4 value=80 time=8
note_on channel=9 note=38 velocity=0 time=37
note_on channel=9 note=38 velocity=38 time=0
note_on channel=9 note=38 velocity=0 time=31
note_o

In [ ]:
#60000 / (BPM * PPQ)(miliseconds)
PPQ=ticks_per_beat=mid.ticks_per_beat
ms_per_tick=(60000 / (95 * PPQ))
print(f' miliseconds per tick = {ms_per_tick}')

#mido.tick2second(tick, ticks_per_beat, tempo)
tick=elapsed_ticks
tempo=631578
total_seconds=tick2second(tick,ticks_per_beat,tempo)
print(total_seconds)

1.3157894736842106
6.001306787499999


In [ ]:
# how to calculate bpm ? How to determine time of note-on?
total_ticks=631578
ticks_per_beat=24
beats_per_quarter=4
quarters=8
print(((60 * 1000000) /total_ticks)) # # One minute is 60 million microseconds.

95.00014250021376


In [ ]:
print(mido.tick2second(elapsed_time,ticks_per_beat,total_ticks))

scale=total_ticks * 1e-6 / ticks_per_beat
print(elapsed_time*scale)

print(total_ticks*(60000 /(95 * 24)))

120.02613575
120.02613575
277.0078947368421


In [ ]:
import mido
totalElapsedTime = mido.tick2second(4561, 24, total_ticks)
print(totalElapsedTime)

360.07840725


# list all files

In [ ]:
print(len(midipaths))
print(len(audiopaths))
print(len(list(set(midipaths))))
print(len(list(set(audiopaths))))

1150
1150
1150
1091


In [ ]:
def get_data_subpaths(datapath):
  folders=[]
  for c in os.listdir(datapath):
    subpath=os.path.join(datapath,c)
    if (os.path.isdir(subpath)):
      #print(f'{subpath} is a folder')
      folders.append(subpath)
    elif (c=="info.csv"):
      #print(f"{subpath} is the annotation file")
      csv_path=subpath
    else:
      #print(f'{c} is an irrelevant file - not of my concern!')
      pass
  return csv_path,folders

def read_csv(csv_path):
  midipaths,audiopaths,bpms,durations,splits,beat_types,sessions=[],[],[],[],[],[],[]
  csv=pd.read_csv(csv_path)  
  #print(csv.shape)
  idxs = csv.index.values.tolist()
  for i,idx in enumerate(idxs):
    #print(f'index {idx}, value {csv.loc[idx,:]}')
    midipaths.append(os.path.join(datapath,csv.loc[idx,"midi_filename"]))
    #audio_filename=csv.loc[idx,"audio_filename"])
    try:
      audiopaths.append(os.path.join(datapath,csv.loc[idx,"audio_filename"]))
    except TypeError:
      audiopaths.append("-")
    bpms.append(csv.loc[idx,"bpm"])
    durations.append(csv.loc[idx,"duration"])
    splits.append(csv.loc[idx,"split"])
    beat_types.append(csv.loc[idx,"beat_type"])
    sessions.append(csv.loc[idx,"session"])
  return midipaths,audiopaths,bpms,durations,splits,beat_types,sessions
  
csv_path,data_folders=get_data_subpaths(datapath)
midipaths,audiopaths,bpms,durations,splits,beat_types,sessions=read_csv(csv_path)
#for subf in data_folders:
#  print(os.listdir(subf))